# Automatyczna klasyfikacja terenu z wykorzystaniem uczenia maszynowego

Zrodlo danych: Sentinel T34UDE_20200815T095039
Kroki wykonane przed analizą:
* Zmiana formatu warstw z jp2 na png ze względu na bezproblemową współpracę z OpenCV
* Klasyfikacja terenu z QGIS oraz wtyczkę QuickOSM
 ** water
 ** forest
 ** farmland
* Dla każdego typu terenu utworzono maskę w formie obrazu PNG o rozdzielczości zgodnej z danymi wejściowymi
* Utworzenie pliku konfiguracyjnego config.ini

#### Wczytanie bibliotek

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from configparser import ConfigParser

In [ ]:
import preprocesing as pre
import helpers as hlp

#### Wczytanie danyc z pliku konfiguracyjnego

In [ ]:
config = ConfigParser()

config.read('config.ini')
input_dir = config['main']['input_dir']# Folder ze zdjęciami z Sentinela
class_file = config['main']['classification_data']# Folder z maskami klas
# Zdjęcia o rozdzielczości 10m składają sie z ponad 100 milionów pikseli
# zatem do analizy wykorzystam tylko jego fragment o rozmiarze dx na dy
# i zaczynający się od piksela (x_star, y_start)
dx = int(config['main']['x_size'])
dy = int(config['main']['y_size'])
x_start = int(config['main']['x_start'])
y_start = int(config['main']['y_start'])
csv_data_file = config['main']['csv_data_file']

#### Przekształcamy dane wejściowe w coś przyjemniejszego do analizy

In [ ]:
data, columns_names = pre.images_to_numpy(input_dir, dx, dy, x_start, y_start)

In [ ]:
hlp.plot_MinMaxAvg(data, columns_names)

In [ ]:
hlp.plot_values_histogram(data, columns_names, ncols=5)

#### Rozkład wartości pikseli wskazuje na występowanie wartości odstających zatem przekształćmy je w następujący sposób: $ x = min(x,\overline{x}+3\sigma_{x}) $ oraz przeskalujmy z wykorzystaniem minmaxscaler z sklearn

In [ ]:
data = pre.remove_outstandings(data)

In [ ]:
hlp.plot_MinMaxAvg(data, columns_names)

In [ ]:
hlp.plot_values_histogram(data, columns_names, ncols=5)

#### Wczytajmy teraz maski klas oraz stwórzmy klasę "other"

In [ ]:
classes, class_names = pre.get_classes(class_file, dx, dy, x_start, y_start)
other = (1 - classes.any(axis=1).astype(int)).reshape(-1,1)
class_names += ['other']
pre.add_classes_to_config(config, class_names)
columns_names += class_names 

In [ ]:
hlp.show_classes_distribution(np.concatenate((classes, other), axis=1),class_names)

In [ ]:
data = np.concatenate((data, classes, other), axis=1)
data = pd.DataFrame(data, columns=columns_names)
data[class_names] = data[class_names].astype('int')

In [ ]:
data.head()

Dane zostały przygotowane zapisujemy je i możemy zająć się klasyfikacją

In [ ]:
data.to_csv(csv_data_file)

#### Wczytujemy nazwy klas

In [ ]:
config = ConfigParser()
config.read('config.ini')
clases_names = list(config['classes'].values())
nr_of_classes = len(clases_names)

#### Dzielimy dane

In [ ]:
X = data.iloc[:,1:-nr_of_classes].to_numpy()
Y = data.iloc[:,-nr_of_classes:].to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

# Klasyfikacja obszaru z wykorzystaniem lasów losowych

#### Trenowanie

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

#### Testowanie

In [ ]:
y_pred_RF = clf.predict(X_test)

In [ ]:
print("Random forest acc: ",accuracy_score(y_test, y_pred_RF))

#### Wizualizacja wyników

In [ ]:
y_pred_RF = clf.predict(X)
y_pred_RF = np.rint(y_pred_RF)
a = y_pred_RF.reshape((dx, dy, nr_of_classes))
b = Y.reshape((dx, dy, nr_of_classes))
hlp.show_target_pred_dif(b,a)

# Klasyfikacja obszaru z wykorzystaniem sieci neuronowych

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten

In [ ]:
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=1000)

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print(f'Deap learning acc: {accuracy}')

#### Wizualizacja wyników

In [ ]:
y_pred_DL = model.predict(X)
y_pred_DL = np.rint(y_pred_DL)
a = y_pred_RF.reshape((dx, dy, nr_of_classes))
b = Y.reshape((dx, dy, nr_of_classes))
hlp.show_target_pred_dif(b,a)

# Klasyfikacja obszaru z wykorzystaniem samoorganizujących się map

In [ ]:
from minisom import MiniSom
x_som, y_som = 5,5
som = MiniSom(x=x_som, y=y_som, input_len=X.shape[1], sigma=1.0, learning_rate=0.5)
som.random_weights_init(X)
som.train_random(X, num_iteration=100000, verbose=False)

#### rysujemy mapę

In [ ]:
fig, ax = plt.subplots()
ax.set_title('SOM')
plt.imshow(som.distance_map())
for (i, j), z in np.ndenumerate(som.distance_map()):
    ax.text(j, i, '{:0.2f}'.format(som.distance_map()[i,j]), ha='center', va='center',color = 'white')

#### Klasyfikujemy
Każdemu punktowi możemy przypisać jeden z neuronów mapy

In [ ]:
y_pred_SOM = [som.winner(x) for x in X]
y_pred_SOM = np.array([i[0]*100 + i[1] for i in y_pred_SOM ])

#### One Hot Encode

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y_pred_SOM = enc.fit_transform(y_pred_SOM.reshape(-1, 1)).toarray()

In [ ]:
xxx = list(range(25))
hlp.show_classes_distribution(y_pred_SOM, xxx)

#### Walidacja

In [ ]:
clusstered = np.zeros((x_som,y_som,3))
matrix_IoU = hlp.metrics_matrix(Y, y_pred_SOM, hlp.IoU)
clusstered[...,2]=matrix_IoU[:,0].reshape((x_som,y_som))
clusstered[...,1]=matrix_IoU[:,1].reshape((x_som,y_som))
clusstered[...,0]=matrix_IoU[:,2].reshape((x_som,y_som))
fig, ax = plt.subplots()
ax.set_title('Intersection over union')
plt.imshow(clusstered)
for (i, j, k), z in np.ndenumerate(clusstered):
    if z > 0.05:
        ax.text(j, i, '{:0.2f}'.format(max(clusstered[i,j,:])), ha='center', va='center',color = 'white')

Kolorami oznaczono klasę którą reprezentują. Słabe wyniki spowodowane są dużo wyższą liczbą otrzymanych klas niż klas które mieliśmy początkowo.

In [ ]:
best_2 = [matrix_IoU[:,i].argsort()[-2:][::-1] for i in range(3)]
frs = [i[0] for i in best_2]
snd = [i[1] for i in best_2]

In [ ]:
Y_present = y_pred_SOM.reshape((dx,dy,y_pred_SOM.shape[1]))

In [ ]:
y_target = Y[...,:-1].reshape((dx,dy,nr_of_classes - 1))
y_present = Y_present[...,frs] + Y_present[...,snd]

In [ ]:
hlp.show_target_pred_dif(y_target,y_present)

# Porównanie skuteczności algorytmów

In [ ]:
y_RF = clf.predict(X)

In [ ]:
hlp.show_classes_distribution(y_RF, class_names)

In [ ]:
y_DL = model.predict(X)

In [ ]:
y_DL = np.rint(y_DL)

In [ ]:
hlp.show_classes_distribution(y_DL, class_names)

In [ ]:
hlp.show_target_pred_dif(y_RF.reshape((dx,dy,nr_of_classes)),y_DL.reshape((dx,dy,nr_of_classes)))

# Analiza wyników